# Train Data

Train a neural network on the images imported in the previous step.

In [2]:
import h5py

In [3]:
hdf_dataset = "imported_dataset.hdf5"

with h5py.File(hdf_dataset, "r") as fid:
    pictures = fid["pictures"].value
    classification = fid["classification"].value

In [4]:
from __future__ import division, print_function, absolute_import

# Import tflearn and some helpers
import tflearn
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

In [ ]:
import sklearn.model_selecti